In [ ]:
from dcicutils import ff_utils

"""When there is a schema change removing a field or an item from enum list
We need to make sure that all items in all environments are compatible with 
the new schema"""

# make a dictionary of schemas, fields, and  excluded enums. If enum check is none
# it will check the existence of field, instead of the enum value
# Example dict
# check_dict = {
#     'FileMicroscopy':{'file_classification':['a'],
#                                },
#     'Biosource':{'biosource_type':['primary cell'],
#                  'description':[],
#                 }
# }

check_dict = {'FileMicroscopy':{'file_classification':['Microscopy']}}


all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']

def check_field(item, field, value):
    # does the item has the field
    current_value = item.get(field)
    # if there is a field value, change it to list format for ease of further operation.
    if current_value:
        if not isinstance(current_value,list):
            current_value = [current_value,]
    else:
        return
    # if value, check for enum, if not, report the field
    if not value:
        print(item['uuid'], 'has deleted field', field)
    else:
        # check if there is a intersection of current and forbiden values
        common = set(value) & set(current_value)
        if common:
            common_print = ','.join(list(common))
            print(item['uuid'], 'has not allowed value(s)', common_print, 'in field', field)
        else:
            return

        
for an_env in all_envs:
    # generate keys
    my_auth = ff_utils.get_authentication_with_server({}, ff_env = an_env)
    print('working on env', an_env)
    
    for a_type in check_dict.keys():
        # get all items
        # in principle we can do a specific search, but if what we are searching for is a field,
        # and the value is empty, it becomes tricky to do the search correctly
        all_items = ff_utils.search_metadata('/search/?limit=all&type='+ a_type, key=my_auth)
        print('checking visible items', a_type, len(all_items))
        # iterate over all items
        for an_item in all_items:
            # iterate over all fields to be checked
            for a_field in check_dict[a_type].keys():
                check_field(an_item, a_field, check_dict[a_type][a_field])

        # do same for deleted items        
        all_del_items = ff_utils.search_metadata('/search/?limit=all&status=deleted&type='+ a_type, key=my_auth)
        print('checking deleted items', a_type, len(all_del_items))
        # iterate over all items
        for del_item in all_del_items:
            # iterate over all fields to be checked
            for a_field in check_dict[a_type].keys():
                check_field(an_item, a_field, check_dict[a_type][a_field])

    print
   